# LangGraph Agent v3 — Complete (Analyze → Search → Enrich → Process)

Final, self-contained agent with LLM decision to enrich or process.

Quick guide:
- State: messages, search_results, optimized_query, decision
- Flow: analyze → search → decide → (enrich | process) → process → END
- Observe prints: [ANALYZE], [SEARCH], [DECIDE], [ROUTE], [ENRICH], [PROCESS], [END]
- The decision is persisted in state.decision and printed after run

In [1]:
import os
from typing import List, Optional, Union, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field


from typing import Annotated
from langgraph.graph import add_messages

from langgraph.graph import StateGraph, END

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_community.utilities import GoogleSearchAPIWrapper

from langchain_community.document_loaders import WebBaseLoader

load_dotenv()

#SERP

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
# GOOGLE_API_KEY =
# GOOGLE_CSE_ID  =  


In [3]:
class SearchResult(BaseModel):
    id: int
    title: str
    snippet: str
    link: str
    full_content: Optional[str] = None

class AgentState(BaseModel):
    messages: Annotated[List[Union[HumanMessage, AIMessage, SystemMessage]], add_messages] = Field(description="The chat history")
    search_results: List[SearchResult] = Field(default_factory=list)
    optimized_query: Optional[str] = None
    decision: Optional[Literal['enrich','process']] = None
    

    class Config:
        arbitrary_types_allowed = True

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
  base_url="https://api.mistral.ai/v1",
  api_key=os.environ["MISTRAL_API_KEY"],
  model="mistral-small-2506" , 
 temperature=0.7
)

In [5]:
###

#llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
search_engine = GoogleSearchAPIWrapper()

C:\Users\PC\AppData\Local\Temp\ipykernel_22252\139544478.py:4: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import GoogleSearchAPIWrapper``.
  search_engine = GoogleSearchAPIWrapper()


In [6]:
def fetch_full_content(url: str, max_length: int = 8000) -> str:
    try:
        docs = WebBaseLoader(web_paths=[url], header_template={"User-Agent":"Mozilla/5.0"}).load()
        if docs and docs[0].page_content.strip():
            t = docs[0].page_content
            return t[:max_length] + ('…' if len(t) > max_length else '')
        return '[Full content unavailable. Using snippet]'
    except Exception as e:
        return f'[Content unavailable: {e}]'

In [7]:
def analyze_query(state: AgentState):
    print("[ANALYZE] Optimizing user query…")
    user_query = next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You rewrite user queries into high-recall google web search queries. Today's date is 2025-09-24. Preserve key entities and constraints (dates, names, locations). Add helpful synonyms if useful. Return ONLY the optimized query with no quotes or extra text an unnecesary symbols."),
        ("human", "Optimize: {q}")
    ])
    q = llm.invoke(prompt.format_messages(q=user_query)).content.strip().replace('"','')
    return {"optimized_query": q}

In [8]:
def perform_search(state: AgentState):
    print("[SEARCH] Running web search…")
    query = state.optimized_query or next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    try:
        raw = search_engine.results(query, num_results=5)
    except Exception:
        raw = [{"title":"Search unavailable","snippet":f"Could not search: {query}","link":"https://example.com"}]
    results = [SearchResult(id=i+1,
                            title=r.get('title','No title'),
                            snippet=r.get('snippet', r.get('description','No content')),
                            link=r.get('link', r.get('url','No link')))
               for i, r in enumerate(raw)]
    return {"search_results": results}

In [9]:

def decide_next(state: AgentState):
    """LLM-backed decision node that writes the decision into state."""
    print("[DECIDE] Choosing 'enrich' vs 'process'…")

    user_query = next((m.content for m in state.messages if isinstance(m, HumanMessage)), 'Unknown')

    p = ChatPromptTemplate.from_messages([
        ('system', "Today is 2025-09-24. Reply ONLY 'enrich' or 'process'. Choose 'process' only if the current snippets clearly, fully in detail answer the query with sufficient detail and sources. If there are not enough info or the question requires details - choose 'enrich', focus on what information the question requires. Be conservative."),
        ('human', 'Query: {q}\nSnippets:\n' + '\n'.join([f'- {r.title}: {r.snippet}' for r in state.search_results]))
    ])
    try:
        d = llm.invoke(p.format_messages(q=user_query)).content.strip().lower()
        decision = 'process' if ('process' in d and 'enrich' not in d) else 'enrich'
    except Exception:
        decision = 'enrich'
    print(f"[DECIDE] Decision: {decision}")
    return {"decision": decision}

In [10]:
def enrich_content(state: AgentState):
    print("[ENRICH] Fetching and extracting relevant page content…")
    if not state.search_results:
        return {}
    user_query = next((m.content for m in state.messages if isinstance(m, HumanMessage)), 'Unknown')
    for r in state.search_results:
        raw = fetch_full_content(r.link)
        print(raw)
        if raw.startswith('[Content unavailable'):
            r.full_content = raw
        else:
            print("-------")
            print(raw)
            print("-------")
            p = ChatPromptTemplate.from_messages([
                ("system", "From the page content, extract only information directly relevant to the user's query. Prefer concrete facts, figures, quotes, and timelines. Include brief inline citations with the source URL (e.g., [source: <URL>]). Ignore irrelevant sections."),
                ("human", "Query: {q}\nTitle: {t}\nURL: {u}\nContent:\n{c}")
            ])
            r.full_content = llm.invoke(p.format_messages(q=user_query, t=r.title, u=r.link, c=raw)).content.strip()
    return {"search_results": state.search_results}

In [11]:

def process_results(state: AgentState):
    print("[PROCESS] Synthesizing final answer…")
    user_query = next((m.content for m in state.messages if isinstance(m, HumanMessage)), 'Unknown')
    if state.search_results:
        blocks = []
        for r in state.search_results:
            snippet = r.snippet or ''
            detail = r.full_content
            blocks.append(f"Result {r.id} | {r.title}\n{snippet}\nSource: {r.link}\n{detail}")
        res = '\n'.join(blocks)
    else:
        res = 'No search results available.'
    prompt = ChatPromptTemplate.from_messages([
        ('system', 'Answer the query using only the provided results. Be detailed, precise, and cite URLs inline (e.g., [source: <URL>]). If evidence is weak or conflicting, say so and avoid speculation.'),
        ('human', 'Query: {q}\n\nResults:\n{res}')
    ])
    ans = llm.invoke(prompt.format_messages(q=user_query, res=res)).content.strip()
    return {"messages": [AIMessage(content=ans)]}



In [12]:

def route_by_decision(state: AgentState) -> Literal['enrich','process']:
    nxt = state.decision or 'enrich'
    print(f"[ROUTE] decide → {nxt}")
    return nxt


In [13]:
workflow = StateGraph(AgentState)
workflow.add_node('optimize', analyze_query)
workflow.add_node('search', perform_search)
workflow.add_node('decide', decide_next)
workflow.add_node('enrich', enrich_content)
workflow.add_node('process', process_results)

workflow.set_entry_point('optimize')


workflow.add_edge('optimize', 'search')
workflow.add_edge('search', 'decide')

workflow.add_conditional_edges('decide', route_by_decision, {'enrich':'enrich','process':'process'})

workflow.add_edge('enrich','process')

workflow.add_edge('process',END)


complete_agent = workflow.compile()

In [14]:
# Test v3
#query = "Who won the Chemistry nobel in 2025?"
query = "What teams were in 2025 NBA playoffs?"
state = AgentState(messages=[HumanMessage(content=query)])
final_state = complete_agent.invoke(state)



[ANALYZE] Optimizing user query…
[SEARCH] Running web search…
[DECIDE] Choosing 'enrich' vs 'process'…
[DECIDE] Decision: enrich
[ROUTE] decide → enrich
[ENRICH] Fetching and extracting relevant page content…
2025 NBA Playoffs | Home | NBA.com

Navigation ToggleHomeTicketsKey Dates2025-26 Regular Season ScheduleNational TV GamesEmirates NBA Cup ScheduleLeague Pass ScheduleCosm ScheduleNBA Cup HomeBracketScheduleStandingsNBA Cup 101FAQKey DatesFeaturedNBA TVHomeTop StoriesFeatures2026 All-StarNBA on Christmas DayEventsKey DatesTransactionsFuture Starts NowHistoryMoreStats HomeDunk ScoreInside the GamePlayersTeamsLeadersStats 101Cume StatsLineups ToolMedia Central Game StatsDraftQuick LinksContact Us2025-26 Regular Season StandingsDivisionConferenceEmirates NBA CupPreseasonAtlanticBoston CelticsBrooklyn NetsNew York KnicksPhiladelphia 76ersToronto RaptorsCentralChicago BullsCleveland CavaliersDetroit PistonsIndiana PacersMilwaukee BucksSoutheastAtlanta HawksCharlotte HornetsMiami HeatOrl

In [15]:
for m in final_state['messages']:
    if isinstance(m, HumanMessage):
        print(f'Human: {m.content}')
    elif isinstance(m, AIMessage):
        print(f'AI: {m.content}')
    else:
        print(m)

Human: What teams were in 2025 NBA playoffs?
AI: Based on the provided results, the teams that were in the 2025 NBA playoffs are as follows:

**From Result 1 (https://www.nba.com/playoffs/2025):**
- Oklahoma City Thunder
- Indiana Pacers
- Golden State Warriors
- Memphis Grizzlies
- Orlando Magic
- Miami Heat
- New York Knicks

**From Result 2 (https://www.espn.com/nba/playoff-bracket):**
- Western Conference:
  - Oklahoma City Thunder
  - Denver Nuggets
  - Los Angeles Clippers
  - Minnesota Timberwolves
  - Golden State Warriors
  - Houston Rockets
  - Los Angeles Lakers
- Eastern Conference:
  - Indiana Pacers
  - New York Knicks
  - Cleveland Cavaliers
  - Boston Celtics
  - Miami Heat
  - Detroit Pistons
  - Orlando Magic

**From Result 4 (https://en.wikipedia.org/wiki/2025_NBA_playoffs) and Result 5 (https://www.nba.com/news/2025-nba-playoffs-schedule):**
- Eastern Conference:
  - Cleveland Cavaliers
  - Boston Celtics
  - New York Knicks
  - Indiana Pacers
  - Milwaukee Bucks
  

In [ ]:
final_state

{'messages': [HumanMessage(content='What teams were in 2025 NBA playoffs?', additional_kwargs={}, response_metadata={}, id='39baea85-cbbb-4f7c-85cb-5168d4437601'),
  AIMessage(content='Based on the provided results, the teams that were in the 2025 NBA playoffs are as follows:\n\n**From Result 1 (https://www.nba.com/playoffs/2025):**\n- Oklahoma City Thunder\n- Indiana Pacers\n- Golden State Warriors\n- Memphis Grizzlies\n- Orlando Magic\n- Miami Heat\n- New York Knicks\n\n**From Result 2 (https://www.espn.com/nba/playoff-bracket):**\n- Western Conference:\n  - Oklahoma City Thunder\n  - Denver Nuggets\n  - Los Angeles Clippers\n  - Minnesota Timberwolves\n  - Golden State Warriors\n  - Houston Rockets\n  - Los Angeles Lakers\n- Eastern Conference:\n  - Indiana Pacers\n  - New York Knicks\n  - Cleveland Cavaliers\n  - Boston Celtics\n  - Miami Heat\n  - Detroit Pistons\n  - Orlando Magic\n\n**From Result 4 (https://en.wikipedia.org/wiki/2025_NBA_playoffs) and Result 5 (https://www.nba.c

: 